In [1]:
import pandas as pd
import plotly
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

In [2]:
# df_agr = pd.read_csv('~/Documents/Energy Research/Value_of_Production_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')
# df_emi = pd.read_csv('~/Documents/Energy Research/Emissions_Agriculture_Agriculture_total_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')
# df_inp = pd.read_csv('~/Documents/Energy Research/Inputs_FertilizersNutrient_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')

In [3]:
shared_id_vars = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit']
necessary_id_vars = ['Area', 'Item', 'Element', 'Unit', 'Year', 'value']

In [4]:
df_configs = {
    'df_agr': {
        'filename': 'Value_of_Production_E_All_Area_Groups_NOFLAG.csv',
        'element': 'Gross Production Value (constant 2014-2016 million US$)',
        'value_name': 'Agr_Production_Value'
    },
    'df_emi': {
        'filename': 'Emissions_Agriculture_Agriculture_total_E_All_Area_Groups_NOFLAG.csv',
        'element': 'Emissions (CO2eq)',
        'value_name': 'Emissions_Value'
    },
    'df_inp': {
        'filename': 'Inputs_FertilizersNutrient_E_All_Area_Groups_NOFLAG.csv',
        'element': 'Agricultural Use',
        'value_name': 'Fertilizer_Value'
    }
}

data = {}
for df_name, config in df_configs.items():
    df = pd.read_csv(config['filename'], encoding='latin1')
    df = df.melt(id_vars = shared_id_vars, var_name = 'Year')
    df = df[necessary_id_vars]
    df = df[df.Element == config['element']]
    df = df.rename(columns={'value': config['value_name']})
    data[df_name] = df

In [5]:
df_agr = data['df_agr']
df_emi = data['df_emi']
df_inp = data['df_inp']

In [6]:
# Add subtotals
necessary_inp_columns=['Area', 'Item', 'Element', 'Unit', 'Year', 'Fertilizer_Value']
df_inp_subtotals = pd.DataFrame(columns=necessary_inp_columns)
for area_year, sub_df in df_inp.groupby(['Area', 'Year']):
    subt = pd.DataFrame([[area_year[0], 'Total Fertilizer', 'Agricultural Use', 'tonnes', area_year[1], sub_df.Fertilizer_Value.sum()]], columns=necessary_inp_columns)
    df_inp = df_inp.append(subt)

# Remove projections
projected_years = ['Y2019', 'Y2030', 'Y2050']
df_emi = df_emi[~df_emi.Year.isin(projected_years)]

In [7]:
# select only aggregates
agr_aggregates = ['Agriculture (PIN)', 'Crops (PIN)', 'Livestock (PIN)']
df_agr = df_agr[df_agr.Item.isin(agr_aggregates)]

# change units from $mm to $s
df_agr['Agr_Production_Value'] = df_agr['Agr_Production_Value'] * 10 ** 6
df_agr['Element'] = 'Gross Production Value (constant 2014-2016 US$)'

<ipython-input-7-6512a5b0b428>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agr['Agr_Production_Value'] = df_agr['Agr_Production_Value'] * 10 ** 6
<ipython-input-7-6512a5b0b428>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agr['Element'] = 'Gross Production Value (constant 2014-2016 US$)'


In [8]:
df_combo = df_agr.merge(df_inp, on=['Area', 'Year'])
df_combo = df_combo.merge(df_emi, on=['Area', 'Year'])

In [9]:
df_combo['Agr Production per Fertilizer Inp'] = df_combo['Agr_Production_Value'].div(df_combo['Fertilizer_Value'], axis=0)
df_combo['Agr Production per Emissions'] = df_combo['Agr_Production_Value'].div(df_combo['Emissions_Value'], axis=0)

In [10]:
# Begin plots

In [11]:
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

In [12]:
# Agriculture production plot

def common_filtering_ag_prod(area):
    plot_output_ag_prod.clear_output()
    
    common_filter_ag_prod = df_agr[df_agr.Area == area]
    
    with plot_output_ag_prod:
        fig_ag_prod = px.line(common_filter_ag_prod, x="Year", y="Agr_Production_Value", color='Item')
        fig_ag_prod.update_layout(yaxis_title='Production Value (2014-2016 USD)',
                      title='Agricultural Production Value')
        fig_ag_prod.show()

def dropdown_area_eventhandler(change):
    common_filtering_ag_prod(change.new)


plot_output_ag_prod = widgets.Output()

# dropdown_area_ag_prod = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))
dropdown_area_ag_prod = widgets.Dropdown(options=sorted(df_agr.Area.unique()))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_ag_prod.observe(dropdown_area_eventhandler, names='value') 
display(dropdown_area_ag_prod)
display(plot_output_ag_prod)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

In [13]:
# Fertilizer use plot

def common_filtering_fert(area):
    plot_output_fert.clear_output()
    
    common_filter_fert = df_inp[df_inp.Area == area]
    
    with plot_output_fert:
        fig_fert = px.line(common_filter_fert, x="Year", y="Fertilizer_Value", color='Item')
        fig_fert.update_layout(yaxis_title='Fertilizer (tonnes)',
                      title='Agricultural Fertilizer Use')
        fig_fert.show()

def dropdown_area_eventhandler(change):
    common_filtering_fert(change.new)


plot_output_fert = widgets.Output()

dropdown_area_fert = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_fert.observe(dropdown_area_eventhandler, names='value')
display(dropdown_area_fert)
display(plot_output_fert)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

In [14]:
# Emissions plot

def common_filtering(area):
    plot_output.clear_output()
    
    common_filter = df_emi[df_emi.Area == area]
    
    with plot_output:
        fig = px.line(common_filter, x="Year", y="Emissions_Value", color='Item')
        fig.update_layout(yaxis_title='Emissions in CO2eq (gigagrams)',
                      title='Agricultural Emissions of CH4 and NO2 in CO2 Equivalent')
        fig.show()

def dropdown_area_eventhandler(change):
    common_filtering(change.new)


plot_output = widgets.Output()

dropdown_area = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area.observe(dropdown_area_eventhandler, names='value')
display(dropdown_area)
display(plot_output)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()